In [1]:
import os
from qiskit import *
from qiskit import QuantumCircuit, Aer
import qiskit.circuit.library as library
import sympy
import numpy as np
from numpy import pi
from scipy import linalg
from functools import reduce
import itertools

from utils.gates import *
import utils.idx as idx
import decomp

import sympy

In [2]:
def run_small():
    fids = []
    for dir in os.listdir('QASMBench/small'):
        try:
            print('='*80)
            u = f'QASMBench/small/{dir}/{dir}.qasm'
            print(dir)
            d = decomp.Decomposer(u, False)
            u = d.unitary

            d.reconstruct_with_CX()
            qc = d.circuit
            backend = Aer.get_backend('unitary_simulator')
            job = execute(qc, backend)
            result = job.result()
            unitary = np.array(result.get_unitary(qc, decimals=10))[:u.shape[0], :u.shape[0]]
            print(fid(u,unitary))
            fids.append((dir, fid(u,unitary)))
            #print('SUCCESS,' if np.all(np.isclose(u, unitary)) else 'FAIL,', 'fidelity:', fid(u, unitary))
            #display(sympy.Matrix(unitary))
            #display(qc.draw(fold=150))
        except Exception as e:
            print(e)
    return fids
#f = run_small()

In [9]:
import os
from functools import reduce
#u = np.kron(Z, np.kron(X,Y))
u = "QASMBench/small/iswap_n2/iswap_n2.qasm"
#u = random_u(2)
d = decomp.Decomposer(u, optimize=False, csdstop=2)
u0 = d.unitary
nbits = int(np.log2(u0.shape[0]))
#display(sympy.Matrix(u0))

print('='*150)

d.reconstruct_with_CX()
print('Done')

size = 2**nbits

backend = Aer.get_backend('unitary_simulator')
job = execute(d.circuit, backend)
result = job.result()
unitary = np.array(result.get_unitary(d.circuit, decimals=10))[:size, :size]
print(fid(u0,unitary))
#display(sympy.Matrix(unitary))
if nbits <= 3:
    display(d.circuit.draw(fold=150))

d = decomp.Decomposer(u, optimize=False)
u0 = d.unitary
nbits = int(np.log2(u0.shape[0]))
#display(sympy.Matrix(u))

print('='*150)

d.reconstruct_with_CU()
print('Done')

size = 2**nbits

backend = Aer.get_backend('unitary_simulator')
job = execute(d.circuit, backend)
result = job.result()
unitary = np.array(result.get_unitary(d.circuit, decimals=10))[:size, :size]
print(fid(u0,unitary))
#display(sympy.Matrix(unitary))
if nbits <= 3:
    display(d.circuit.draw(fold=150))

Running CSDfactor... 0.00s
Total multiplexed single-qubit gates: 3
Running extract_unitaries... 0.00s
running reconstruct_with_CX... 0.01s
cu identity gates removed by extract_unitaries: 1
total cu gates after extract_unitaries: 5
total cx gates after convert_to_cx: 10
total cnx gates: 10
Done
0.3173140730682316


┌───┐┌───┐┌───┐┌───┐    ┌───┐     ┌───┐          ┌───┐┌───┐┌───┐    ┌───┐                                                    ┌───┐┌───┐»
q14378_0: ┤ C ├┤ X ├┤ B ├┤ X ├────┤ A ├─────┤ C ├──────────┤ X ├┤ B ├┤ X ├────┤ A ├────────────────■────────────────────────────────■──┤ C ├┤ X ├»
          └───┘└─┬─┘└───┘└─┬─┘    └───┘     └───┘┌───┐     └─┬─┘└───┘└─┬─┘    └───┘         ┌───┐  │  ┌───┐┌───┐┌───┐┌───┐ ┌───┐    │  └───┘└─┬─┘»
q14378_1: ──■────┼─────────┼──────────────────■──┤ X ├──■────┼─────────┼─────────────────■──┤ X ├──┼──┤ C ├┤ X ├┤ B ├┤ X ├─┤ A ├────┼────■────┼──»
          ┌─┴─┐  │         │  ┌────────────┐┌─┴─┐└───┘┌─┴─┐  │         │  ┌───────────┐┌─┴─┐└───┘┌─┴─┐└───┘└─┬─┘└───┘└─┬─┘┌┴───┴─┐┌─┴─┐┌─┴─┐  │  »
      a8: ┤ X ├──■─────────■──┤ P(0.57241) ├┤ X ├─────┤ X ├──■─────────■──┤ P(1.0722) ├┤ X ├─────┤ X ├───────■─────────■──┤ P(0) ├┤ X ├┤ X ├──■──»
          └───┘               └────────────┘└───┘     └───┘               └───────────┘└───┘     └───┘                    └──────┘└───┘└───┘     »
«          ┌───┐┌───┐    ┌───┐    ┌───┐          ┌───┐┌───┐┌───┐    ┌───┐               
«q14378_0: ┤ B ├┤ X ├────┤ A ├────┤ C ├──────────┤ X ├┤ B ├┤ X ├────┤ A ├───────────────
«          └───┘└─┬─┘    └───┘    └───┘┌───┐     └─┬─┘└───┘└─┬─┘    └───┘          ┌───┐
«q14378_1: ───────┼─────────────────■──┤ X ├──■────┼─────────┼──────────────────■──┤ X ├
«                 │  ┌───────────┐┌─┴─┐└───┘┌─┴─┐  │         │  ┌────────────┐┌─┴─┐└───┘
«      a8: ───────■──┤ P(3.7806) ├┤ X ├─────┤ X ├──■─────────■──┤ P(0.28684) ├┤ X ├─────
«                    └───────────┘└───┘     └───┘               └────────────┘└───┘

Running CSDfactor... 0.00s
Total multiplexed single-qubit gates: 3
Running extract_unitaries... 0.00s
running reconstruct_with_CU... 0.05s
Done
1.0


┌─────────┐     ┌─────────┐                ┌─────────┐     ┌─────────┐     
q_0: ┤ Unitary ├─────┤ Unitary ├──────────■─────┤ Unitary ├─────┤ Unitary ├─────
     └────┬────┘┌───┐└────┬────┘┌───┐┌────┴────┐└────┬────┘┌───┐└────┬────┘┌───┐
q_1: ─────■─────┤ X ├─────■─────┤ X ├┤ Unitary ├─────■─────┤ X ├─────■─────┤ X ├
                └───┘           └───┘└─────────┘           └───┘           └───┘

We can decompose the multi-controlled U gate into three $C^{n-1}X$ gates (and then to $C^{n-1}Z$ gates) through the following:

In [4]:
from qiskit.circuit import Gate, ControlledGate
import qiskit.circuit
from qiskit.circuit.library import ZGate, PhaseGate
from qiskit.quantum_info import Operator

ccz = ZGate().control(2)

u = Operator([[1,0],[0,1]]).to_instruction().control(2)
u.name = 'U'
qc = QuantumCircuit(3)
qc.append(u, [0,1,2])
display(qc.draw())

qc = QuantumCircuit(3)
qc.append(Gate(name='C', num_qubits=1, params=[]), [2])
qc.toffoli(0,1,2)
qc.append(Gate(name='B', num_qubits=1, params=[]), [2])
qc.toffoli(0,1,2)
qc.append(Gate(name='A', num_qubits=1, params=[]), [2])
qc.toffoli(0,1,2)
qc.append(Gate(name='U^t(0,-a,0)', num_qubits=1, params=[]), [2])
qc.toffoli(0,1,2)
qc.append(Gate(name='U(0,-a,0)', num_qubits=1, params=[]), [2])
qc.append(PhaseGate(pi/2).control(2), [0,1,2])
qc.draw(fold=-1)

q_0: ─────■─────
          │     
q_1: ─────■─────
     ┌────┴────┐
q_2: ┤ Unitary ├
     └─────────┘

q_0: ───────■─────────■─────────■───────────────────■────────────────■───────
            │         │         │                   │                │       
q_1: ───────■─────────■─────────■───────────────────■────────────────■───────
     ┌───┐┌─┴─┐┌───┐┌─┴─┐┌───┐┌─┴─┐┌─────────────┐┌─┴─┐┌───────────┐ │P(π/2) 
q_2: ┤ C ├┤ X ├┤ B ├┤ X ├┤ A ├┤ X ├┤ U^t(0,-a,0) ├┤ X ├┤ U(0,-a,0) ├─■───────
     └───┘└───┘└───┘└───┘└───┘└───┘└─────────────┘└───┘└───────────┘

where $U = e^{ia}AXBXC$ (N&C page 180). The last five gates correspond to a controlled $e^{i \alpha}$ phase applied to the target bit.

From here, this can easily be converted to CZ gates with $A' = HA, B' = HBH, C' = CH, U' = UH$

In [5]:
# TODO: recognize that this is a tensor product of two single-qubit gates
u = np.kron(Z,Y)
acc = np.identity(4)
for m in decomp.CSDfactor(u):
    for ex in decomp.extract_unitaries(m)[1]:
        display(sympy.Matrix(ex))
        acc = acc @ ex
display(sympy.Matrix(acc))

AttributeError: module 'decomp' has no attribute 'CSDfactor'

In [ ]:
u = random_u(8)
csd = decomp.CSDfactor(u)
for m in csd:
    decomp.extract_unitaries(m)

In [ ]:
a,A,B,C = ABC(twoq)

In [ ]:
sympy.Matrix(np.exp(1j*a) * (A @ np.kron(X,X) @ B @ np.kron(X,X) @ C))

In [ ]:
from qiskit.compiler import transpile
qc = QuantumCircuit.from_qasm_file('QASMBench/small/dnn_n2/dnn_n2.qasm')
transpile(qc, basis_gates=['u3', 'cx'], optimization_level=3).draw()

In [ ]:
sympy.Matrix(rx(np.pi/3))

In [ ]:
sympy.Matrix(np.kron(ry(np.pi/3), ry(-np.pi/3)) @ CNOT @ np.kron(ry(-np.pi/3), ry(np.pi/3)))

In [ ]:
# want to get this matrix:
sympy.Matrix([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0.8, 0.6],
    [0, 0, 0.6, 0.8]
])

In [ ]:
sympy.Matrix(np.kron(rz(np.pi/3), rz(-np.pi/3)))

In [ ]:
linalg.sqrtm(CNOT)

In [ ]:
# goal: apply rotation of top qubit onto bottom qubit - needed for my construction
# can it be done with rz? use formula for controlled rz, but do the A and B matrices on the other qubit?

In [ ]:
pauli_decomp(np.kron(X,Z))

[0j, 0j, (1+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j]

In [ ]:
pauli_reconstruct(pauli_decomp(np.kron(X,Z)), 2)

array([[ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j],
       [ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j]])

In [ ]:
np.kron(X,Z)

array([[ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
       [ 0.+0.j, -0.+0.j,  0.+0.j, -1.+0.j],
       [ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j,  0.+0.j, -0.+0.j]])